<a href="https://colab.research.google.com/github/FranciscoNav/springboard/blob/main/ComputerVision/CNNs_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lecture Refrence:
1. https://youtu.be/_L2uYfVV48I
2. https://youtu.be/3V9YFPfgyAY
Notes: the code below is just notes and practice code from the lecture referenced above. It is not an actual project.

In [ ]:
from ast import increment_lineno
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
# import kagglehub
# Download latest version
# path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")
# print("Path to dataset files:", path)

warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
# Organize data into train, valid test dir
os.chdir('data/dogs-vs-cats')
if os.path.isdir('train/dog') is False:
  os.makedirs('train/dog')
  os.makedirs('train/cat')
  os.makedirs('valid/dog')
  os.makedirs('valid/cat')
  os.makedirs('test/dog')
  os.makedirs('test/cat')
  for c in random.sample(glob.glob('cat*'),500):
    shutil.move(c, 'train/cat')
  for c in random.sample(glob.glob('dog*'),500):
    shutil.move(c, 'train/dog')
  for c in random.sample(glob.glob('cat*'),100):
    shutil.move(c,'valid/cat')
  for c in random.sample(glob.glob('dog*'),100):
    shutil.move(c,'valid/dog')
  for c in random.sample(glob.glob('cat*'),50):
    shutil.move(c,'test/cat')
  for c in random.sample(glob.glob('dog*'),50):
    shutil.move(c,'test/cat')

os.chdir('../../')

In [ ]:
train_path = 'data/dogs-vs-cats/train'
valid_path = 'data/dogs-vs-cats/valid'
test_path = 'data/dogs-vs-cats/test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat','dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vg16.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat','dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat','dog'], batch_size=10, shuffle=False)

In [ ]:
assert train_batches.n == 1000
assert valid_batches.n == 200
assert test_batches.n == 100
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classess == 2

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding ='same', input_shape=(224,224,3))
    # this will cut image in half
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding ='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
# since we are only doing cats, and dogs...we could use binary crossentropy and Sigmoid rather than softmax above
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', mmetrics=[])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)

In [ ]:
test_batches.classes
predictions = model.predict(x=test_batches, verbose=0)
np.round(predictions)